In [10]:
# Function to count the number of URLs in the file
def count_urls(file_path):
    url_count = 0
    
    # Open and read the file
    with open(file_path, 'r') as file:
        for line in file:
            # Count non-empty lines (assuming each line contains one URL)
            if line.strip():  # Strip any whitespace and check if the line is not empty
                url_count += 1
                
    return url_count

# Specify the path to your text file
file_path = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20240930_urls.txt"

# Call the function and print the result
print(f'Total number of URLs: {count_urls(file_path)}')

Total number of URLs: 1372


In [11]:
import os

# Function to list and count files in a folder, excluding certain file types
def list_downloaded_files(folder_path):
    downloaded_files = []

    # Walk through the directory structure
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            # Exclude files ending with '.download.iocs'
            if not file_name.endswith('.download.iocs'):
                downloaded_files.append(file_name)

    # Return the list of files and the total count
    return downloaded_files, len(downloaded_files)

# Specify the path to the folder containing downloaded files
folder_path = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20241008_downloads"

# Call the function and get the list of files and their count
downloaded_files, total_count = list_downloaded_files(folder_path)

# Print the list of files and the total count
print("List of downloaded files (SHA256 hash names):")
#for file_name in downloaded_files:
#    print(file_name)

print(f"\nTotal number of downloaded files: {total_count}")


List of downloaded files (SHA256 hash names):

Total number of downloaded files: 899


In [12]:
import json

# Function to identify successful downloads, filtering by download size
def identify_successful_downloads(file_path):
    successful_downloads = []
    
    # Open and read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object
                download_info = json.loads(line)
                
                # Check if the download status is 200 (successful) and size is 20KB or more
                if download_info.get("download_status") == 200 and download_info.get("download_size", 0) >= 20000:
                    successful_downloads.append({
                        "url": download_info.get("url"),
                        "download_sha256": download_info.get("download_sha256"),
                        "download_status": download_info.get("download_status"),
                        "download_size": download_info.get("download_size"),
                        "download_ts": download_info.get("download_ts")
                    })
            except json.JSONDecodeError:
                # Handle case where the line is not a valid JSON (optional)
                print(f"Error decoding JSON for line: {line.strip()}")
    
    return successful_downloads

# Specify the path to your JSONL file
file_path = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20241008_downloads.jsonl"

# Call the function and get the successful downloads
successful_downloads = identify_successful_downloads(file_path)

# Print out the results
#for download in successful_downloads:
#    print(f"URL: {download['url']}, SHA256: {download['download_sha256']}, Size: {download['download_size']} bytes, Status: {download['download_status']}")

# Print the total count of successful downloads with size >= 20KB
print(f"\nTotal number of successful downloads (size >= 20KB): {len(successful_downloads)}")



Total number of successful downloads (size >= 20KB): 1146


In [14]:
import json

# Function to identify successful downloads
def identify_successful_downloads(file_path):
    successful_downloads = []
    
    # Open and read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object
                download_info = json.loads(line)
                
                # Check if the download status is 200 (successful)
                if download_info.get("download_status") == 200:
                    successful_downloads.append({
                        "url": download_info.get("url"),
                        "download_sha256": download_info.get("download_sha256"),
                        "download_status": download_info.get("download_status"),
                        "download_size": download_info.get("download_size"),
                        "download_ts": download_info.get("download_ts")
                    })
            except json.JSONDecodeError:
                # Handle case where the line is not a valid JSON (optional)
                print(f"Error decoding JSON for line: {line.strip()}")
    
    return successful_downloads

# Specify the path to your JSONL file
file_path = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20241008_downloads.jsonl"

# Call the function and print the successful downloads
successful_downloads = identify_successful_downloads(file_path)

# Print out the results
#for download in successful_downloads:
#    print(f"URL: {download['url']}, SHA256: {download['download_sha256']}, Status: {download['download_status']}")

# Print the total count of successful downloads
print(f"\nTotal number of successful downloads: {len(successful_downloads)}")



Total number of successful downloads: 1186
